In [ ]:
# load dataset
import dataloader
import utilities
import transcribe
import matplotlib.pyplot as plt
import numpy as np
import madmom
import os
from datetime import datetime
import features
import pickle

parameters = {"chroma_type":"crp",
        "vocabulary":"majmin_sevenths",  # triads, triads_extended, majmin_sevenths
        "eval_scheme":"majmin_sevenths",
        "hprs":False,
        "prefilter":"median",
        "prefilter_length":17,
        "postfilter":"hmm",
        "transition_prob":0.2,
        "save_transcription":False
}

PATH = "/home/max/ET-TI/Masterarbeit/mirdata/"
output_path = "/home/max/ET-TI/Masterarbeit/results/"
timestamp = datetime.now().strftime('%m_%d_%Y_%H_%M_%S')
output_path += timestamp
os.mkdir(output_path)
if parameters.get('save_transcription',False):
    os.mkdir(output_path+"/chromagrams")
    os.mkdir(output_path+"/transcriptions")
results= {}
results["parameters"] = parameters
for fold in range(1,9):
    for name in ["rwc_popular","beatles"]:
        dataset = dataloader.MIRDataset(name,basepath=PATH,split_nr=fold)
        for track_id in dataset.getTrackList():
            try:
                name = dataset.getTitle(track_id).split("_-_")[-1]
                track_results = {}
            except KeyError:
                print(f"error loading {name}.. skipping song")
                continue
            audiopath,(ref_intervals,ref_labels) = dataset[track_id]

            time_vector,signal = utilities.loadAudio(audiopath)

            ## HPS ##
            if parameters.get("hprs",False):
                y_harm,y_perc,y_res = transcribe.harmonicPercussiveResidualSeparation(signal,beta=3,n_fft=4096)
                signal = madmom.audio.signal.Signal(y_harm, sample_rate=signal.sample_rate, num_channels=1, start=signal.start, stop=signal.stop)

            if parameters.get("chroma_type","crp") == "crp":
                t_chroma,chroma = features.crpChroma(signal)
            elif parameters.get("chroma_type") == "dcp":
                t_chroma,chroma = features.deepChroma(signal,split_nr=fold)
            else:
                t_chroma,chroma = features.cqtChroma(signal)

            est_intervals, est_labels = transcribe.transcribeChromagram(t_chroma,chroma,**parameters)
            score,mean_seg_score = transcribe.evaluateTranscription(est_intervals,est_labels,ref_intervals,ref_labels,parameters.get("eval_scheme","majmin"))
            print(f"Track: {name}, {parameters.get('eval_scheme','majmin')}: {round(score*100,1)}, segmentation: {round(mean_seg_score*100,1)}")
            track_results[parameters.get('eval_scheme','majmin')] = score
            track_results["segmentation"] = mean_seg_score
            if parameters.get('save_transcription',False):
                track_results["t_chroma"] = t_chroma
                track_results["chroma"] = chroma
                track_results["est_intervals"] = est_intervals
                track_results["est_labels"] = est_labels
                utilities.saveTranscriptionResults(output_path,name,t_chroma,chroma,est_intervals,est_labels,ref_intervals,ref_labels)

            results[name] = track_results 
            break
        continue
f = open(output_path+f"/{parameters.get('eval_scheme','majmin')}_{parameters.get('chroma_type')}.pkl","wb")
pickle.dump(results, f)
f.close()
